<a href="https://colab.research.google.com/github/abhigadde/Cal_Newport_Question_Repository/blob/main/Cal_Newport_Question_List.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal is to go through Cal Newport's Deep Work podcast and pull all timestamps and questions into a single excel file so users can easily see all the 1500+ questions Cal has answered over his 200+ episodes and only listen to the questions that pertain most to them without wasting time going through each podcast episode.

In [ ]:
pip install spotipy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


For security reasons, I can't share my client secret. So to run the code below and use the Web API, start by creating a Spotify user account (Premium or Free). To do that, simply sign up at www.spotify.com.

When you have a user account, go to the Dashboard page at the Spotify Developer website and, if necessary, log in. Accept the latest Developer Terms of Service to complete your account set up. Then all you need to is create an app and put that client ID and client secret below. This is all completely free and should take under 10 minutes.

In [ ]:
import spotipy
import re
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
cid = 'YOUR CLIENT ID'
secret = 'YOUR CLIENT SECRET'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
#pull all the fields of interest from Spotify using the spotipy library

uri = '0e9lFr3AdJByoBpM6tAbxD' #ID for cal newports podcast. This is the last part of the url one puts into google chrome to access the spotify webpage
total_episode_count = sp.show_episodes(uri, limit = 50, offset = 0, market = 'US').get('total') 
#These are 3 fields of interest we want to output
show_descriptions = []
episode_url = []
episode_number = []
    
for i in range(0,total_episode_count,50): #spotify sadly limits calls to size 50 so need a for loop here
    episode_list = sp.show_episodes(uri, limit = 50, offset = i, market = 'US')
    item_list = episode_list.get('items')
    
    for j in range(0,len(item_list),1):
        show_descriptions.append(item_list[j]['description'])
        episode_url.append(item_list[j]['external_urls'].values())
        episode_number.append(re.split(':', item_list[j]['name'])[0])
        #print(j)
    #print(i)


#Note excluding episodes 9 & 10 (in addition to episodes 1-8 since they don't have timestamps) since even though those 2 episodes have timestamps, 
#they are in front of the description  whereas it seems all later episodes have the timestamp at the end of the description
#Lastly, I am also reversing the order of all the lists so counts episodes in ascending rather than descending order.

episode_url.reverse()
episode_url_truncated = episode_url[10:]

episode_number.reverse()
episode_number_truncated = episode_number[10:]

show_descriptions_with_timestamps = show_descriptions[:len(show_descriptions)-10]
show_descriptions_with_timestamps.reverse()

In [ ]:
#Note since we want to delimit using "]"(done later in code) our first line will have a lot of excess text we will want to remove.
#Therefore, we need to process the first line text seperately.

first_line = []
for i in range(0,len(show_descriptions_with_timestamps),1):
    a = re.split('\]', show_descriptions_with_timestamps[i])
    #print (*a,sep='\n')
    first_line.append(a[0])

In [ ]:
#Have 5 different rules in place based on how cal updated his podcast descriptions over time.

first_line_truncate = []
for i in range(0,41,1):
    first_line_truncate.append(re.split('\*', first_line[i])[-1])

for i in range(41,49,1): #this is by far the hardest segment of shows to split out the first line for. Luckily not too many episodes
    first_line_truncate.append(re.split('\*|-', first_line[i])[-1])
    
for i in range(49,156,1):
    first_line_truncate.append(re.split('podcast.', first_line[i])[-1])

for i in range(156,168,1):
    first_line_truncate.append(re.split('b2rkctfj', first_line[i])[-1])
    
for i in range(168,len(first_line),1): #I assume he will keep the same description format in the future. If he doesn't this needs to be updated
    first_line_truncate.append(re.split('calnewportmedia', first_line[i])[-1])
    

In [ ]:
#Now we can combine the specially processed first lines with all the other lines. We will also include episode link and number here.

show_desc_df = pd.DataFrame(columns=['timestamp', 'link','episode name'])
for i in range(0,len(show_descriptions_with_timestamps),1):
    df2 = pd.DataFrame({'timestamp':[first_line_truncate[i]], 'link': episode_url_truncated[i], 'episode name': episode_number_truncated[i]})
    show_desc_df = show_desc_df.append(df2, ignore_index=True)
    
    df3 = pd.DataFrame({'timestamp':re.split('\]', show_descriptions_with_timestamps[i])[1:][:-1], 'link': 'N/A', 'episode name': episode_number_truncated[i]})
    df3['link'] = df3['link'].replace(['N/A'],episode_url_truncated[i])
    show_desc_df = show_desc_df.append(df3, ignore_index=True)

In [ ]:
#Last touchups to timestamps

#Write ] to every value in timestamp column cause delimiter above removed it
show_desc_df['timestamp'] = show_desc_df['timestamp'].astype(str) + ']'

#Remove some extraneous repeated words
pattern = '|'.join(['TECHNOLOGY QUESTIONS', 'TECHNOLOGY QUESTION', 'DEEP LIFE QUESTIONS', 'DEEP WORK QUESTIONS','WORK QUESTIONS', 'QUESTIONS:', '\* ','  ','- ',' - '])
show_desc_df['timestamp'] = show_desc_df['timestamp'].str.replace(pattern,'', regex = True)

In [ ]:
#Write out final dataframe to csv and download it
from google.colab import files

show_desc_df.to_csv('final_output.csv', encoding='utf-8-sig')
files.download('final_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>